# Multiworker TF Experimental

In [2]:
import json
import os
import sys

Since this is experimental for a single machine, multiworker refers to multi-thread, not multiple machines. So disabling the GPUs

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [4]:
os.environ.pop('TF_CONFIG', None)

Be sure that the current directory is on python's path. This allows the notebook to import the files written by `%%writefile` later.


In [5]:
if '.' not in sys.path:
  sys.path.insert(0, '.')

Now import TensorFlow.

In [6]:
import tensorflow as tf

### Dataset and model definition

Next create an `mnist.py` file with a simple model and dataset setup. This python file will be used by the worker-processes in this tutorial:

In [7]:
%%writefile mnist.py

import os
import tensorflow as tf
import numpy as np

def mnist_dataset(batch_size):
  (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
  # The `x` arrays are in uint8 and have values in the range [0, 255].
  # You need to convert them to float32 with values in the range [0, 1]
  x_train = x_train / np.float32(255)
  y_train = y_train.astype(np.int64)
  train_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, y_train)).shuffle(60000).repeat().batch(batch_size)
  return train_dataset

def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.Input(shape=(28, 28)),
      tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
      metrics=['accuracy'])
  return model

Writing mnist.py


Try training the model for a small number of epochs and observe the results of a single worker to make sure everything works correctly. As training progresses, the loss should drop and the accuracy should increase.

In [8]:
import mnist

batch_size = 64
single_worker_dataset = mnist.mnist_dataset(batch_size)
single_worker_model = mnist.build_and_compile_cnn_model()
single_history = single_worker_model.fit(single_worker_dataset, epochs=30, steps_per_epoch=70)

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/30
70/70 [==============================] - 4s 44ms/step - loss: 2.2795 - accuracy: 0.1451
Epoch 2/30
70/70 [==============================] - 3s 43ms/step - loss: 2.2068 - accuracy: 0.2766
Epoch 3/30
70/70 [==============================] - 3s 43ms/step - loss: 2.1214 - accuracy: 0.5063
Epoch 4/30
70/70 [==============================] - 3s 44ms/step - loss: 2.0097 - accuracy: 0.6334
Epoch 5/30
70/70 [==============================] - 3s 43ms/step - loss: 1.8781 - accuracy: 0.6873
Epoch 6/30
70/70 [==============================] - 3s 43ms/step - loss: 1.7274 - accuracy: 0.7198
Epoch 7/30
70/70 [==============================] - 3s 44ms/step - loss: 1.5557 - accuracy: 0.7444
Epoch 8/30
70/70 [==============================] - 3s 44ms/step - loss: 1.3944 - accuracy: 0.7533
Epoch 9/30
70/70 [==============================] - 3s 44ms/step - loss: 1.2144 - accuracy: 0.7860
Epoch 10/30
70/70 [=========================

## Multi-worker Configuration

In worker specify localhost with different ports. 
This will be automatically craeted while running

In [9]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']
    },
    'task': {'type': 'worker', 'index': 0}
}

In [10]:
json.dumps(tf_config)

'{"cluster": {"worker": ["localhost:12345", "localhost:23456"]}, "task": {"type": "worker", "index": 0}}'

In [11]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO


In [12]:
%%writefile main.py

import os
import json

import tensorflow as tf
import mnist

per_worker_batch_size = 64
tf_config = json.loads(os.environ['TF_CONFIG'])
num_workers = len(tf_config['cluster']['worker'])

strategy = tf.distribute.MultiWorkerMirroredStrategy()

global_batch_size = per_worker_batch_size * num_workers
multi_worker_dataset = mnist.mnist_dataset(global_batch_size)

with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = mnist.build_and_compile_cnn_model()


multi_worker_model.fit(multi_worker_dataset, epochs=30, steps_per_epoch=70)

Writing main.py


In [13]:
%%bash
ls *.py

main.py
mnist.py


In [14]:
os.environ['TF_CONFIG'] = json.dumps(tf_config)

Now, you can launch a worker process that will run the `main.py` and use the `TF_CONFIG`:

In [15]:
# first kill any previous runs
%killbgscripts

All background processes were killed.


In [16]:
%%bash --bg
python main.py &> job_0.log

Starting job # 0 in a separate thread.


In [17]:
import time
time.sleep(10)

Now look what's been output to the worker's logfile so far:

In [18]:
%%bash
cat job_0.log

2021-04-01 00:44:28.033755: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-01 00:44:30.221644: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-01 00:44:30.222650: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-01 00:44:30.233887: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-01 00:44:30.233946: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (f232e3cbd766): /proc/driver/nvidia/version does not exist
2021-04-01 00:44:30.234952: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-01 00:44:30.235577: I tensorflow/compiler/jit/xla_gpu_device.cc:99] 

update the index to 1.
For the second process to start

In [19]:
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)

Now launch the second worker. This will start the training since all the workers are active (so there's no need to background this process):

In [20]:
%%bash
python main.py

Epoch 1/30
70/70 [==============================] - 11s 144ms/step - loss: 2.2887 - accuracy: 0.1172
Epoch 2/30
70/70 [==============================] - 10s 144ms/step - loss: 2.2340 - accuracy: 0.3429
Epoch 3/30
70/70 [==============================] - 10s 144ms/step - loss: 2.1702 - accuracy: 0.5294
Epoch 4/30
70/70 [==============================] - 10s 145ms/step - loss: 2.0951 - accuracy: 0.6360
Epoch 5/30
70/70 [==============================] - 10s 143ms/step - loss: 1.9983 - accuracy: 0.6968
Epoch 6/30
70/70 [==============================] - 10s 144ms/step - loss: 1.8730 - accuracy: 0.7197
Epoch 7/30
70/70 [==============================] - 10s 145ms/step - loss: 1.7210 - accuracy: 0.7451
Epoch 8/30
70/70 [==============================] - 10s 145ms/step - loss: 1.5565 - accuracy: 0.7483
Epoch 9/30
70/70 [==============================] - 10s 145ms/step - loss: 1.3822 - accuracy: 0.7688
Epoch 10/30
70/70 [==============================] - 10s 146ms/step - loss: 1.2209 - accura

2021-04-01 00:44:38.094939: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-01 00:44:40.266198: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-01 00:44:40.267235: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-01 00:44:40.277995: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-01 00:44:40.278075: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (f232e3cbd766): /proc/driver/nvidia/version does not exist
2021-04-01 00:44:40.279048: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-01 00:44:40.279533: I tensorflow/compiler/jit/xla_gpu_device.cc:99] 

Now if you recheck the logs written by the first worker you'll see that it participated in training that model:

In [21]:
%%bash
cat job_0.log

2021-04-01 00:44:28.033755: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-01 00:44:30.221644: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-01 00:44:30.222650: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-01 00:44:30.233887: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-01 00:44:30.233946: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (f232e3cbd766): /proc/driver/nvidia/version does not exist
2021-04-01 00:44:30.234952: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-01 00:44:30.235577: I tensorflow/compiler/jit/xla_gpu_device.cc:99] 

Running slower than the single thread. Because of same machine process overload. 

In [22]:
# Delete the `TF_CONFIG`, and kill any background tasks so they don't affect the next section.
os.environ.pop('TF_CONFIG', None)
%killbgscripts

All background processes were killed.
